In [1]:
%pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 3.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch as th

In [3]:
th.cuda.is_available()

True

In [50]:
import sys
sys.path.append("src/")
import yaml
import numpy as np
import pandas as pd
import torch_tensorrt
from dataset import PlanetDataset
from torch.utils.data import DataLoader
import torchmetrics
from torchmetrics.classification import BinaryAUROC, BinaryAccuracy

from utils import preprocess_image

from src.model.torch import ModelTorch
from src.utils import MAX_UINT8

BATCH_SIZES = [1,2,4,8]
DEVICE = "cuda"


In [54]:
def get_metrics(model):
    acc_fn = BinaryAccuracy(task="multilabel", num_labels=17)
    auc_fn = BinaryAUROC(task="multilabel", num_labels=17)
    accuracies = []
    aucs = []
    for images, labels in loader:
        pred = model(images.to("cuda"))
        acc = acc_fn(pred.to("cpu"),labels)
        auc = auc_fn(pred.to("cpu"),labels)
        accuracies.append(acc)
        aucs.append(auc)
    return {"auc": np.mean(aucs), "acc": np.mean(accuracies)}

In [6]:
dataset = PlanetDataset(
    df=pd.read_csv("/workspace/data/valid_df.csv"),
    image_folder="/workspace/data/train-jpg/"
)
loader = DataLoader(
    dataset=dataset,
    batch_size=8,
    num_workers=4,
    shuffle=False,
    drop_last=False,
)


In [7]:
records = []
with open("./config/torch.yaml") as fp:
    cfg_torch = yaml.safe_load(fp)

cfg_torch["device"] = DEVICE
model_torch = ModelTorch(cfg_torch)

In [57]:
trt_model = torch_tensorrt.compile(
    model_torch.model,                                       
    inputs = [
        torch_tensorrt.Input(
            (8, 3, MAX_UINT8, MAX_UINT8)
        )
    ], 
    enabled_precisions = th.float32,                # <- изменения здесь
    workspace_size = 1 << 30,                     
)
print(get_metrics(trt_model))

{'auc': 0.9417781, 'acc': 0.91496164}


In [58]:
trt_model = torch_tensorrt.compile(
    model_torch.model,                                       
    inputs = [
        torch_tensorrt.Input(
            (8, 3, MAX_UINT8, MAX_UINT8)
        )
    ], 
    enabled_precisions = th.float16,                # <- изменения здесь
    workspace_size = 1 << 30,                     
)
print(get_metrics(trt_model))

{'auc': 0.94175446, 'acc': 0.91496164}
